# Interactive calibration

In [43]:
import math
import os
from os.path import join
import re

from ipywidgets import Checkbox, FloatSlider, IntSlider
import ipywidgets as widgets
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot
from scipy.interpolate import UnivariateSpline, interp1d
from scipy.signal import savgol_filter as savitzky_golay

In [44]:
from quest_pp_calibration.tvregdiff import TVRegDiff # https://github.com/stur86/tvregdiff

# Set-up data folder

In [45]:
date = 170112
pp_pth = f'../../data/extract_shot/{date}'

dr = os.listdir(pp_pth)
filelist = sorted([i for i in dr])
pp = 8

# Enter valid calibration factors

In [46]:
# 02_calc_pumping_time/calc_pumping_time_170123.ipynb
kqms = {8:2.91e+22} 
tau = {8:31.16}

# surface area (Vacuum 2016)
A = {8:7.5e-03}

# Apply calibration, get the permeated flux

\begin{equation}
\Gamma_{pdp} = Sa \cdot \left( I - I_0 + \tau\frac{dI}{dt}\right)
\end{equation}
Where $Sa$ - calibration coefficient to convert QMS current into particle flux for steady state pressures ($\tau\frac{dI}{dt} \ll I$).

In [47]:
latest_results = {}

def pp_analyse(i, f1, f2, win_sma, win_sg1, tosave, win_sg2, shot_start, w, w_range, s):
    global latest_results
    
    df = pd.read_csv(join(pp_pth,filelist[i]), sep=r'\s+', header=None)
    num_cols = df.select_dtypes('number').columns
    x = df[num_cols[1]]

    # x の値でフィルター
    filtered_df = df[x <= 5000]
    x = filtered_df[num_cols[1]]
    y = filtered_df[num_cols[2]]

    xx = np.linspace(x.min(),x.max(),len(x))
    fy = interp1d(x,y)
    yy = fy(xx)

    index = np.abs(xx).argmin()
    offset = yy[index]
    yy = (yy - offset)*kqms[8]/A[8]

    # 微分前のスムージング
    yy_smoothed = pd.Series(yy).rolling(window=win_sma, center=True, min_periods=1).mean().values
    yy_smoothed = savitzky_golay(yy_smoothed,win_sg1,2)

    # 微分計算
    freq1 = f2*(10**f1)
    deriv = TVRegDiff(
        data = yy_smoothed, itern = 200, alph = freq1, dx=xx[1]-xx[0],
        ep=1e-2, scale='small', plotflag=False, diagflag=False
    )
    signal_raw = yy_smoothed + tau[pp] * deriv

    # --- spline 計算ここから ---
    ref_value = np.abs(signal_raw).max()
    if ref_value > 0:
        exponent = int(math.floor(math.log10(ref_value)))
    else:
        exponent = -12
    signal_exponent = 10 ** exponent
    signal_noisy = signal_raw/signal_exponent

    # 1. 全ての重みを1にする
    weights = np.ones_like(xx)

    # 2. 区間（w_range）のインデックスを特定
    quiet_zone_indices = np.where(xx < w_range)

    # 3. その区間の重みを大きくする
    weights[quiet_zone_indices] = w

    # 4. 重みを適用してスプラインを計算
    spline_with_weight = UnivariateSpline(xx, signal_noisy, w=weights, s=s)

    signal_spline = spline_with_weight(xx)*signal_exponent
    # --- spline 計算ここまで ---

    # 放電開始時刻より後でスプラインの適応
    signal = np.where(
        xx >= shot_start,
        signal_spline,
        yy_smoothed
    )
    
    # 全体のスムージング
    signal = savitzky_golay(signal,win_sg2,2)

    plotdata = [
        go.Scatter(
            x = xx[::3],
            y = yy[::3],
            mode = 'markers',
            name = 'raw',
            marker = dict(
                size = 5,
                color = 'purple',
                opacity = 0.5
            )
        ),
        go.Scatter(
            x = xx,
            y = yy_smoothed,
            mode = 'lines',
            name = 'fitting',
            line = dict(
                color = 'darkorange',
            )
        ),
        go.Scatter(
            x = xx,
            y = signal_raw,
            mode = 'lines',
            name = 'calibration',
            line = dict(
                color = 'green',
            )
        ),
        go.Scatter(
            x = xx,
            y = signal,
            mode = 'lines',
            name = f'calibration(spline)'
        ),
    ]

    try:
        number = re.search(r"(\d{5})", filelist[i]).group(1)
    except AttributeError:
        print("5桁の数字が見つかりません")

    layout = dict(
        title = 'Permeation {}'.format(number),
        xaxis = dict(
            title = 'time, s',        
        ),
        yaxis = dict(title = 'Permeation flux'),
        yaxis2 = dict(
            title = 'tau*dI/dt',
            overlaying = 'y',
            side = 'right'
        )
    )        

    fig = go.Figure(data=plotdata,layout=layout)
    config = {
        'scrollZoom': False,
        'editable': True,
             }

    iplot(fig,config=config)
    
    if tosave:
        spth = f'../../data/calibrated/{date}'
        fpth = join(spth,'{}_PDP{}_calibrated_f1-{}_f2-{}_winsma-{}_winsg1-{}_winsg2-{}_w{}-range{}_s{}.txt'.format(
            number,
            pp,
            f1,
            f2,
            win_sma,
            win_sg1,
            win_sg2,
            w,
            w_range,
            s
        ))     

        np.savetxt(
            fpth,
            np.array([xx,signal]).T
        )
        print('save')

    latest_results = {
        'xx': xx,
        'yy': yy,
        'signal': signal,
        'deriv': deriv,
        'index': index,
        'a': kqms[8]/A[8],
        'shot_start': shot_start,
    }

In [ ]:
i = IntSlider(min=0,max=len(filelist)-1,value=0,description='i')
f1 = IntSlider(min=-10,max =10,value=-2,step=1,description='f1')
f2 = FloatSlider(min=0,max=10,value = 1,step=.1,description='f2')
win_sma = IntSlider(min=3,max=30,value=30,description='win_sma')
win_sg1 = IntSlider(min=3,max=100,value=71,description='win_sg1')
win_sg2 = IntSlider(min=3,max=100,value=7,description='win_sg2')
shot_start = FloatSlider(min=0,max=10,value = 1.5, step=.1, description='shot_start')
w = IntSlider(min=1,max=100,value=5,description='w')
w_range = IntSlider(min=1,max=500,value=5,description='w_range')
s = IntSlider(min=1,max=100,value=2,description='s')

tosave = Checkbox(value=False,description='Save signal',disabled=False,)
ui = widgets.VBox([
    widgets.HBox([i,tosave]),
    widgets.HBox([
        widgets.VBox([f1,f2,win_sma,win_sg1,win_sg2,]),
        widgets.VBox([shot_start,w, w_range,s])
    ]),
])

w = widgets.interactive_output(
    pp_analyse,
    {
        'i':i,'f1':f1,'f2':f2,'win_sg1':win_sg1,'win_sma':win_sma,
        'tosave':tosave,'win_sg2':win_sg2, 'shot_start': shot_start, 'w':w, 'w_range':w_range, 's':s
    }
)

display(ui,w)

Output()